In [8]:
import xarray as xr
import numpy as np
import pandas as pd
from xarray_selafin.xarray_backend import SelafinAccessor

ds = xr.open_dataset("./wind.slf", engine="selafin")
era5_ds = xr.open_dataset(r'./out.nc')

In [9]:
ds

<xarray.Dataset> Size: 12MB
Dimensions:  (time: 97, node: 10363)
Coordinates:
    x        (node) float32 41kB ...
    y        (node) float32 41kB ...
  * time     (time) datetime64[ns] 776B 2019-10-29T23:00:00 ... 2019-11-02T23...
Dimensions without coordinates: node
Data variables:
    X        (time, node) float32 4MB ...
    Y        (time, node) float32 4MB ...
    P        (time, node) float32 4MB ...
Attributes:
    title:       ATHABASCA RIVER
    language:    en
    float_size:  4
    endian:      >
    params:      (1, 0, 0, 0, 0, 0, 0, 2590, 0, 1)
    ipobo:       [    1     2     3 ... 10361 10362 10363]
    ikle2:       [[8124 8122 8052]\n [7871 7790 7791]\n [3878 4035 3871]\n .....
    variables:   {'X': ('WIND ALONG X', 'M/S'), 'Y': ('WIND ALONG Y', 'M/S'),...
    date_start:  (2019, 10, 29, 23, 0, 0)

In [10]:
np.shape(ds.time.values)

(97,)

In [11]:
np.shape(ds.x.values)

(10363,)

In [12]:
# np.shape(era5_ds.u10.values)
era5_ds["latitude"].values

array([45. , 44.9, 44.8, 44.7, 44.6, 44.5, 44.4, 44.3, 44.2, 44.1, 44. ,
       43.9, 43.8, 43.7, 43.6, 43.5, 43.4, 43.3, 43.2, 43.1, 43. , 42.9,
       42.8, 42.7, 42.6, 42.5])

In [13]:
from scipy.interpolate import griddata
time_steps = len(ds.time.values)
lat_era5  = era5_ds["latitude"].values
lon_era5  = era5_ds["longitude"].values
lons,lats = np.meshgrid(lon_era5, lat_era5)
wind_x_output = np.zeros_like(ds.X.values)
wind_y_output = np.zeros_like(ds.Y.values)
pressure_output = np.zeros_like(ds.P.values)
x_coord = ds.x.values
y_coord = ds.y.values
wind_x_era5 = era5_ds.u10.values
wind_y_era5 = era5_ds.v10.values
wind_p_era5 = era5_ds.sp.values

##### for testing
# wind_u_data = wind_x_era5[0,:,:]
# wind_v_data = wind_y_era5[0,:,:]
# sp_data = wind_p_era5[0,:,:]
# grid_out = np.concatenate([x_coord.reshape(-1,1), y_coord.reshape(-1,1)], axis=1)
# grid_in = np.concatenate([lons.reshape(-1,1), lats.reshape(-1,1)], axis=1)
# u10_out = griddata(grid_in, np.array(wind_u_data).flatten(), grid_out, method='nearest')
# print(np.shape(u10_out))


for time in range(time_steps):
    wind_u_data = wind_x_era5[time,:,:]
    wind_v_data = wind_y_era5[time,:,:]
    sp_data = wind_p_era5[time,:,:]
    grid_out = np.concatenate([x_coord.reshape(-1,1), y_coord.reshape(-1,1)], axis=1)
    grid_in = np.concatenate([lons.reshape(-1,1), lats.reshape(-1,1)], axis=1)
    u10_out = griddata(grid_in, np.array(wind_u_data).flatten(), grid_out, method='nearest')
    wind_x_output[time,:] = u10_out
    v10_out = griddata(grid_in, np.array(wind_v_data).flatten(), grid_out, method='nearest')
    wind_y_output[time,:] = v10_out
    sp_out = griddata(grid_in, np.array(sp_data).flatten(), grid_out, method='nearest')
    pressure_output[time,:] = sp_out
    print("Finished: "+str(time))

# Creating a minimal dataset
ds_new = xr.Dataset(
    {
        "WINDX": (("time", "node"), wind_x_output),
        "WINDY": (("time", "node"), wind_y_output),
        "PATM": (("time", "node"), pressure_output),
        # Add other variables as needed
    },
    coords={
        "x": ("node", ds.x.values), 
        "y": ("node", ds.y.values), 
        "time": ds.time.values,
        # Add "ikle2" or "ikle3" as required by your mesh
    }
)
ds_new.attrs['ikle2'] = ds.ikle2

# Writing to a SELAFIN file
ds_new.selafin.write("wind_era5.slf")


Finished: 0
Finished: 1
Finished: 2
Finished: 3
Finished: 4
Finished: 5
Finished: 6
Finished: 7
Finished: 8
Finished: 9
Finished: 10
Finished: 11
Finished: 12
Finished: 13
Finished: 14
Finished: 15
Finished: 16
Finished: 17
Finished: 18
Finished: 19
Finished: 20
Finished: 21
Finished: 22
Finished: 23
Finished: 24
Finished: 25
Finished: 26
Finished: 27
Finished: 28
Finished: 29
Finished: 30
Finished: 31
Finished: 32
Finished: 33
Finished: 34
Finished: 35
Finished: 36
Finished: 37
Finished: 38
Finished: 39
Finished: 40
Finished: 41
Finished: 42
Finished: 43
Finished: 44
Finished: 45
Finished: 46
Finished: 47
Finished: 48
Finished: 49
Finished: 50
Finished: 51
Finished: 52
Finished: 53
Finished: 54
Finished: 55
Finished: 56
Finished: 57
Finished: 58
Finished: 59
Finished: 60
Finished: 61
Finished: 62
Finished: 63
Finished: 64
Finished: 65
Finished: 66
Finished: 67
Finished: 68
Finished: 69
Finished: 70
Finished: 71
Finished: 72
Finished: 73
Finished: 74
Finished: 75
Finished: 76
Finished:

SERAFIN REQUEST ERROR: The variable "WINDX" is not known (lang=en)
SERAFIN REQUEST ERROR: The variable "WINDY" is not known (lang=en)
SERAFIN REQUEST ERROR: The variable "PATM" is not known (lang=en)


Finished: 89
Finished: 90
Finished: 91
Finished: 92
Finished: 93
Finished: 94
Finished: 95
Finished: 96


In [14]:
np.shape(pressure_output)

(97, 10363)